In [1]:
import pandas as pd 
import numpy as np 
import sklearn.model_selection as sk
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import os
from argparse import ArgumentParser
import logging
from typing import NoReturn
from collections import Counter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.model_selection as sk
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
import evaluation_scripts.eval_passengers_up as eval

In [3]:
TRAIN_BUS_CSV_PATH = "data/train_bus_schedule.csv"
X_PASSENGER = "data/X_passengers_up.csv"
X_TRIP = "data/X_trip_duration.csv"
ENCODER = "windows-1255"
RANDOM_STATE = 42

In [4]:
train_bus = pd.read_csv(TRAIN_BUS_CSV_PATH, encoding=ENCODER)
x_passenger = pd.read_csv(X_PASSENGER, encoding=ENCODER)
x_trip_duration = pd.read_csv(X_TRIP, encoding=ENCODER)

In [ ]:
train_bus = pd.read_csv(TRAIN_BUS_CSV_PATH, encoding=ENCODER)
x_trip = pd.read_csv(X_TRIP, encoding=ENCODER)


In [7]:
lines_for_baseline = train_bus["trip_id_unique"].drop_duplicates().sample(frac=0.10,
                                                                        random_state=RANDOM_STATE)
dur_remaining = train_bus[~train_bus["trip_id_unique"].isin(lines_for_baseline)]

In [11]:
lines_for_xboost = dur_remaining["trip_id_unique"].drop_duplicates().sample(frac=0.28,random_state=RANDOM_STATE)

In [12]:
dur_remaining = train_bus[~train_bus["trip_id_unique"].isin(lines_for_xboost)]

In [19]:
def __creating_labels(dur_baseline):
    min_max_time = dur_baseline.groupby("trip_id_unique")["arrival_time"].agg(
        {"min", "max"}).reset_index()
    min_max_time["max"] = pd.to_datetime(min_max_time["max"])
    min_max_time["min"] = pd.to_datetime(min_max_time["min"])
    min_max_time["delta"] = (min_max_time["max"] - min_max_time["min"]) / pd.Timedelta(1, "m")
    min_max_time["delta"] = round(min_max_time["delta"], 2)
    return min_max_time[["trip_id_unique", "delta"]]


In [20]:
dur_labels = __creating_labels(dur_remaining)

# # 2. preprocess the training set
# logging.info("preprocessing train...")
# X_train, X_test, y_train, y_test = preprocessing_baseline(dur_baseline, dur_labels)


/var/folders/24/vfbkhj7j0t142rz214mskjkr0000gn/T/ipykernel_74156/212821949.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  min_max_time["max"] = pd.to_datetime(min_max_time["max"])
/var/folders/24/vfbkhj7j0t142rz214mskjkr0000gn/T/ipykernel_74156/212821949.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  min_max_time["min"] = pd.to_datetime(min_max_time["min"])


In [22]:
f_station_cnt = dur_remaining.groupby("trip_id_unique")["trip_id_unique_station"].nunique().to_frame(
    "station_cnt")
f_total_passenger = dur_remaining.groupby("trip_id_unique")["passengers_up"].sum().to_frame(
    "total_passenger")
f_mean_passenger = dur_remaining.groupby("trip_id_unique")["passengers_up"].mean().to_frame(
    "mean_passenger")
f_mean_passenger_c = dur_remaining.groupby("trip_id_unique")["passengers_continue"].mean().to_frame(
    "mean_passenger_c")
f_start_time = dur_remaining.groupby("trip_id_unique")["arrival_time"].min().to_frame("start_time")
f_start_time['start_time'] = pd.to_datetime(f_start_time['start_time']).dt.hour
features = pd.concat(
    [f_station_cnt, f_total_passenger, f_mean_passenger, f_mean_passenger_c, f_start_time],
    axis=1)
features = features.merge(dur_remaining[["trip_id_unique", "cluster", "direction", "mekadem_nipuach_luz"]],on="trip_id_unique")

/var/folders/24/vfbkhj7j0t142rz214mskjkr0000gn/T/ipykernel_74156/2488329175.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  f_start_time['start_time'] = pd.to_datetime(f_start_time['start_time']).dt.hour


In [24]:
label_encoder = LabelEncoder()
features['cluster'] = label_encoder.fit_transform(features['cluster'])

In [25]:
words_weight = ['אצ"ל',
 'ביאליק',
 'ההגנה',
 'חזון',
 'לוינסקי',
 'סוקולוב',
 'עקיבא',
 'קניון',
 'רבי',
 'רכבת',
 'אצ"ל',
 'ביאליק',
 'בלפור',
 'גשר',
 'ההגנה',
 'המלך',
 "ז'בוטינסקי",
 'חזון',
 'יוספטל',
 'כצנלסון',
 'לוינסקי',
 'סוקולוב',
 'עקיבא',
 'קניון',
 'רבי',
 'רוטשילד']

In [30]:
station_concat = dur_remaining.groupby("trip_id_unique")["station_name"].agg(lambda x: ', '.join(x)).reset_index()

In [31]:
for i, word in enumerate(words_weight, start=1):
    station_concat[f'x_{i}'] = station_concat['station_name'].str.contains(word, regex=False).astype(int)


In [33]:
del station_concat['station_name']
features = features.merge(station_concat,on = "trip_id_unique")
features = features.drop_duplicates()

In [36]:
dur_remaining

In [40]:
!pip install geopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 597.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 4.7 MB/s eta 0:00:00


In [41]:
import pandas as pd
from geopy.distance import geodesic

In [42]:
def calculate_approx_line_length(group):
    coords = list(zip(group['latitude'], group['longitude']))
    total_length = 0.0
    for i in range(len(coords) - 1):
        total_length += geodesic(coords[i], coords[i + 1]).km
    return total_length

# Group by 'line_id' and calculate approximate line length
line_lengths_approx = dur_remaining.groupby('trip_id_unique').apply(calculate_approx_line_length).reset_index(name='line_length_approx')


/var/folders/24/vfbkhj7j0t142rz214mskjkr0000gn/T/ipykernel_74156/2280330949.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  line_lengths_approx = dur_remaining.groupby('trip_id_unique').apply(calculate_approx_line_length).reset_index(name='line_length_approx')


In [45]:
features = features.merge(line_lengths_approx,on = "trip_id_unique")

In [53]:
dur_labels['']

,trip_id_unique,delta
0,110005c,97.00
1,110007c,76.00
2,110008c,80.00
3,110009c,82.00
4,110014c,113.00
...,...,...
4328,421861a,94.22
4329,421880a,72.00
4330,421882a,103.00
4331,421931a,83.00


In [54]:
X_train, X_test, y_train, y_test = sk.train_test_split(features.drop(columns = ["trip_id_unique"]), dur_labels.drop(columns = ["trip_id_unique"]), train_size=0.75,
                                                        random_state=RANDOM_STATE)

In [55]:
def xg_boost(X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.Series,
             y_test: pd.Series):
    results = []

    # Define parameter grids to iterate over
    parameters = [
        {'max_depth': 3, 'learning_rate': 0.1, 'n_estimators': 100},
        {'max_depth': 5, 'learning_rate': 0.1, 'n_estimators': 100},
        {'max_depth': 5, 'learning_rate': 0.05, 'n_estimators': 100},
        {'max_depth': 5, 'learning_rate': 0.1, 'n_estimators': 200},
        {'max_depth': 7, 'learning_rate': 0.1, 'n_estimators': 100}
    ]

    for params in parameters:
        # Create XGBoost model
        model = xgb.XGBRegressor(objective='reg:squarederror', eval_metric='rmse', **params)

        # Train the model
        model.fit(X_train, y_train)

        # Predict on test set
        y_pred = model.predict(X_test)

        # Calculate RMSE
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        results.append((params, rmse))

    # Return results for further analysis or selection
    return results

In [56]:
result = xg_boost(X_train, X_test, y_train, y_test )

/opt/anaconda3/envs/iml.env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/anaconda3/envs/iml.env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/anaconda3/envs/iml.env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/anaconda3/envs/iml.env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6

In [57]:
result

[({'max_depth': 3, 'learning_rate': 0.1, 'n_estimators': 100},
  70.30254649230075),
 ({'max_depth': 5, 'learning_rate': 0.1, 'n_estimators': 100},
  79.17302176967863),
 ({'max_depth': 5, 'learning_rate': 0.05, 'n_estimators': 100},
  80.67238379498458),
 ({'max_depth': 5, 'learning_rate': 0.1, 'n_estimators': 200},
  79.86132873106735),
 ({'max_depth': 7, 'learning_rate': 0.1, 'n_estimators': 100},
  81.16479731329005)]